**Prophet** is a forecasting procedure that was published by Facebook for quick and efficient forecasting! To learn more about the package: https://facebook.github.io/prophet/

## I. Install Prophet

**1. Install Pystan** <br> Pystan is a package that is required to install and run Prophet. For more information about Pystan installation, see this link: https://pystan.readthedocs.io/en/latest/installation_beginner.html

In [ ]:
#conda install pystan #install using anaconda command
#pip install pystan #install using your laptop's command prompt / terminal

**2. Install Plotly** <br> Plotly is a graphing dependancy that is referenced by the Prophet package. You will not need plotly to execute forecasting with Prophet, but installing the package will prevent error messages from occuring when the prophet package is installed.

In [ ]:
conda install -c plotly plotly=4.5.2 #install using anaconda command
# pip install plotly==4.5.2 #install using your laptop's command prompt / terminal

**3. Install the Prophet Package**

In [ ]:
conda install fbprophet #install using anaconda command
#pip install fbprophet #install using your laptop's command prompt / terminal

## II. Import Packages

In [ ]:
import pandas as pd
from fbprophet import Prophet
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import math
import holidays

## III. Review Prophet Functionality

**Prophet Inputs:** a dataframe with two columns: ds and y. <br>
**Input Format:** 
- The ds (datestamp) column should be of a format expected by Pandas, ideally YYYY-MM-DD for a date or YYYY-MM-DD HH:MM:SS for a timestamp. 
- The y column must be numeric, and represents the measurement we wish to forecast.

**Load Time Series** <br>
We will start exploring with prophet using the prophet example time series: Peyton Manning wikipedia page views. <br>
Source: https://github.com/facebook/prophet/blob/master/examples/example_wp_log_peyton_manning.csv

In [ ]:
wiki_df = pd.read_csv('Peyton_Manning.csv')

The dataframe has two features:
1. **ds:** the date of the recorded viewcount
2. **y:** the log of the views for that day

In [ ]:
wiki_df.head()

**Clean Time Series Data Format**
Note that the loaded time series is not in the correct date format required by Prophet. We need to modify the time series format because the **ds** feature is currently an   **object** type

In [ ]:
wiki_df.info()

In [ ]:
wiki_df.ds = pd.to_datetime(wiki_df.ds)

**Review the updated data**

In [ ]:
wiki_df.info()

In [ ]:
wiki_df.head()

## Visualize the Data
**Plot the Time Series with Matplotlib**

In [ ]:
plt.figure(figsize=(15,10))
wiki_df.y.plot()

## Transform the Data

**Ensure the Data is Centered Around 0 (Stationary)**<br> We start by taking the natural log of the data. 

In [ ]:
wiki_df.head(3)

In [ ]:
stationary = wiki_df
stationary.y = np.log10(stationary.y)

In [ ]:
stationary.head(3)

In [ ]:
plt.figure(figsize=(15,10))
stationary.y.plot()

**Removing Trends by Taking the Difference of Data**

In [ ]:
diff = stationary.y.diff(periods = 1)
stationary.y = diff
stationary.head(3)

In [ ]:
stationary = stationary.drop(stationary.index[0])

In [ ]:
stationary.head(3)

In [ ]:
plt.figure(figsize=(15,10))
stationary.y.plot()

**Review one year of the transformed time series.**

In [ ]:
stationary[0:360]

In [ ]:
plt.figure(figsize=(15,10))
stationary[0:360].y.plot()

## Forecasting & Prediction w/ Default Prophet Functionality
Creating a forecast without modifying the time series or adjusting any hyperparameters. We will start by re-loading the data to remove any previous transformations.

**Defining historical (training) and future (test) data**

In [ ]:
wiki_df = pd.read_csv('Peyton_Manning.csv')
wiki_df.ds = pd.to_datetime(wiki_df.ds)

**Create a Prediction Dataframe**<br>
The prediction dataframe will store the historical (training) data to be used when creating the model. We will remove the the last **prediction_size** observations from the dataset for model testing and evaluation.

In [ ]:
prediction_size = 360
train_df = wiki_df[:-prediction_size]
train_df.tail(n=3)

**Create a Test Dataframe**<br> containing the 360 observations that will be used for future observations & testing.

In [ ]:
test_df = wiki_df[len(wiki_df)-prediction_size:]

In [ ]:
test_df.tail(n=3)

**Creating a Prophet Model** <br> Initiate a prophet model by creating an object to store the Prophet function.<br>
We will use the two dataframes just created to make our model:<br>**train_df**: 2544 historical observations<br>**test_df**: 360 historical observations

In [ ]:
m = Prophet()

Fit the prophet model to the dataset

In [ ]:
m.fit(train_df)

**Create an empty future dataframe to store predictions**<br>Create a dataframe to store the historical data and the model predictions

In [ ]:
future = m.make_future_dataframe(periods=360)
future.tail()

**Predict the 360 days into the future**

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

**Plotting the Forecast** <br>
We have now predicted a year of Peyton Manning's wikipedia page views based on the historical pageviews using Prophet! <br> We will plot the results to see the overall forecast.

In [ ]:
fig1 = m.plot(forecast)

**Review the Components of the Model**

In [ ]:
fig2 = m.plot_components(forecast)

## Identifying Changepoints in the Model

In [ ]:
from fbprophet.plot import add_changepoints_to_plot
plt.figure(figsize=(15,10))
fig = m.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), m, forecast)

## Forecast Quality
Calculate the error for the 360 days predicted.

In [ ]:
print(', '.join(forecast.columns))

**Save the results in one dataframe**

In [ ]:
forecast.head(3)

In [ ]:
results = forecast.set_index('ds')[['yhat']]

In [ ]:
results = results.join(wiki_df.set_index('ds'))

In [ ]:
results.head(3)

In [ ]:
results = results.dropna()

In [ ]:
MAE_1 = mean_absolute_error(results.y, results.yhat)
print('Mean Absolute Error:', MAE_1)

In [ ]:
MSE_1 = mean_squared_error(results.y, results.yhat)
print('Mean Squared Error:', MSE_1)

## Transform the Model w/ Hyperparameters

**Identifying and Fitting a Model w/ Changepoints**

In [ ]:
from fbprophet.plot import add_changepoints_to_plot
fig = m.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), m, forecast)

In [ ]:
#Specify the location of changepoints
m = Prophet(changepoints=['2014-01-01'])
forecast = m.fit(train_df).predict(future)
fig = m.plot(forecast)

In [ ]:
results_cp = forecast.set_index('ds')[['yhat']]
results_cp = results_cp.join(wiki_df.set_index('ds'))
results_cp = results_cp.dropna()
results_cp.head(3)

In [ ]:
MAE_CP = mean_absolute_error(results_cp.y, results_cp.yhat)
MSE_CP = mean_squared_error(results_cp.y, results_cp.yhat)
print('ChangePoint Mean Absolute Error:', MAE_CP, 'Original Mean Absolute Error:', MAE_1)
print('ChangePoint Mean Squared Error:', MSE_CP, 'Original Mean Squared Error:', MSE_1)

**Fitting a Model Accounting for Standard Holidays**

In [ ]:
list(pd.DatetimeIndex(wiki_df.ds).year.unique())

In [ ]:
years = list(pd.DatetimeIndex(wiki_df.ds).year.unique())

In [ ]:
import holidays
holidays.US()
for ptr in holidays.US(years = years).items(): 
    print(ptr) 

In [ ]:
m = Prophet()
m.add_country_holidays(country_name='US')

In [ ]:
forecast = m.fit(train_df).predict(future)

In [ ]:
m.train_holiday_names #Run this after the model has been fit

In [ ]:
results_holiday = forecast.set_index('ds')[['yhat']]
results_holiday = results_holiday.join(wiki_df.set_index('ds'))
results_holiday = results_holiday.dropna()
results_holiday.head(3)

In [ ]:
MAE_holiday = mean_absolute_error(results_holiday.y, results_holiday.yhat)
MSE_holiday = mean_squared_error(results_holiday.y, results_holiday.yhat)
print('Holiday Mean Absolute Error:', MAE_holiday, 'Original Mean Absolute Error:', MAE_1)
print('Holiday Mean Squared Error:', MSE_holiday, 'Original Mean Squared Error:', MSE_1)

**Fitting a Model Accounting for Problem-Specific Holidays**

In [ ]:
playoffs = pd.DataFrame({
  'holiday': 'playoff',
  'ds': pd.to_datetime(['2008-01-13', '2009-01-03', '2010-01-16',
                        '2010-01-24', '2010-02-07', '2011-01-08',
                        '2013-01-12', '2014-01-12', '2014-01-19',
                        '2014-02-02', '2015-01-11', '2016-01-17',
                        '2016-01-24', '2016-02-07']),
  'lower_window': 0,
  'upper_window': 1,
})
superbowls = pd.DataFrame({
  'holiday': 'superbowl',
  'ds': pd.to_datetime(['2010-02-07', '2014-02-02', '2016-02-07']),
  'lower_window': 0,
  'upper_window': 1,
})
holidays = pd.concat((playoffs, superbowls))

In [ ]:
holidays.head()

In [ ]:
m = Prophet(holidays=holidays)
forecast = m.fit(train_df).predict(future)

In [ ]:
forecast[(forecast['playoff'] + forecast['superbowl']).abs() > 0][
        ['ds', 'playoff', 'superbowl']][-10:]

In [ ]:
# Python
fig = m.plot_components(forecast)

In [ ]:
results_playoff = forecast.set_index('ds')[['yhat']]
results_playoff = results_playoff.join(wiki_df.set_index('ds'))
results_playoff = results_playoff.dropna()
results_playoff.head(3)

In [ ]:
MAE_playoff = mean_absolute_error(results_playoff.y, results_playoff.yhat)
MSE_playoff = mean_squared_error(results_playoff.y, results_playoff.yhat)
print('Playoff Mean Absolute Error:', MAE_holiday, 'Original Mean Absolute Error:', MAE_1)
print('Playoff Mean Squared Error:', MSE_holiday, 'Original Mean Squared Error:', MSE_1)

# Now Try on your Own!
We will download historical stock information and try to predict the next 5 days of the stock's performance. We will start with apple, but you can download a different company by searching them on Yahoo Finance and downloading the 'Historical Data' information.

In [ ]:
df = pd.read_csv('AAPL.csv')

In [ ]:
df.head()

In [ ]:
stock = df[['Date','Close']]

In [ ]:
stock.shape

## Split Train & Test Data Before Analysis

In [ ]:
prediction_size = 360
train_df = stock[:-prediction_size]
train_df.columns= ['ds','y']
train_df.tail(n=3)

In [ ]:
test_df = stock[len(stock)-prediction_size:]

## Analysis and Modeling

In [ ]:
stock.Close.plot()

**Create your stock forecast with Prophet!**

In [ ]:
m = Prophet()
m.fit(train_df)
future = m.make_future_dataframe(periods=360)
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

**How do you do compared to the market?**

In [ ]:
stock.columns = ['ds','y']

In [ ]:
results_stock = forecast.set_index('ds')[['yhat']]
results_stock = results_stock.join(stock.set_index('ds'))
results_stock = results_stock.dropna()
results_stock.tail(3)

In [ ]:
MAE_Stock = mean_absolute_error(results_stock.y, results_stock.yhat)
print('Mean Absolute Error:', MAE_Stock)
MSE_Stock = mean_squared_error(results_stock.y, results_stock.yhat)
print('Mean Squared Error:', MSE_Stock)